In [8]:
pip install pyaudio

   ---------------------------------------- 0.0/164.1 kB ? eta -:--:--
   -- ------------------------------------- 10.2/164.1 kB ? eta -:--:--
   ---------------------- ----------------- 92.2/164.1 kB 1.1 MB/s eta 0:00:01
   ---------------------------------------- 164.1/164.1 kB 1.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
import openai
import pyaudio
import speech_recognition as sr
from io import BytesIO
from dotenv import load_dotenv, find_dotenv

_=load_dotenv(find_dotenv())

client = openai.Client()
recognizer = sr.Recognizer()

In [13]:
def grava_audio():
    with sr.Microphone() as source:
        print('Ouvindo...')
        recognizer.adjust_for_ambient_noise(source,duration=1)
        audio = recognizer.listen(source)
        return audio
    
def transcricao_audio(audio):
    wav_data = BytesIO(audio.get_wav_data())
    wav_data.name = 'audio.wav'
    transcricao = client.audio.transcriptions.create(
        model = 'whisper-1',
        file = wav_data,
        response_format = 'text',
        language = 'pt'
        )
    return transcricao

def resposta_chat(mensagens):
    respostas = client.chat.completions.create(
        messages = mensagens,
        model = 'gpt-3.5-turbo-0125',
        max_tokens = 1000,
        temperature = 0,
        stream = True,
    )
    
    print('Assistant:', end = '')
    texto_completo = ''
    for stream_respostas in respostas:
        texto = stream_respostas.choices[0].delta.content
        if texto:
            texto_completo += texto
            print(texto, end = '')
    print()
    mensagens.append({'role': 'assistant', 'content': texto_completo})
    return mensagens

In [18]:
if __name__ == '__main__':
    print('Bem vindo ao ChatBot')
    mensagens = []
    while True:
        audio = grava_audio()
        transcricao = transcricao_audio(audio)
        input_user = f'User: {transcricao}'
        mensagens.append({'role':'user','content':input_user})
        mensagens = resposta_chat(mensagens)


Bem vindo ao ChatBot
Ouvindo...
Assistant:1 mais 1 é igual a 2.
Ouvindo...
Assistant:De nada! Estou aqui para ajudar. Se tiver mais alguma dúvida, é só perguntar.
Ouvindo...


KeyboardInterrupt: 

'Quanto é 1 mais 1?\n'